# Process calcium imaging data with DataJoint Elements

This notebook will walk through processing two-photon calcium imaging data collected
from ScanImage, Scanbox, Nikon NIS, or PrairieView. Later, you'll pict from either
Suite2p or CaImAn for analysis.

The DataJoint Python API and Element Calcium Imaging offer a lot of features to support collaboration, automation, reproducibility, and visualization. For more information on these topics, please visit our documentation: 
 
- [DataJoint Core](https://datajoint.com/docs/core/): General principles

- DataJoint [Python](https://datajoint.com/docs/core/datajoint-python/) and
  [MATLAB](https://datajoint.com/docs/core/datajoint-matlab/) APIs: in-depth reviews of
  specifics

- [Element Calcium Imaging](https://datajoint.com/docs/elements/element-calcium-imaging/):
  A modular pipeline for 2p data analysis

## Setup - Working Directory

To run the workflow, we need to properly set up the DataJoint configuration. The configuration can be saved in a local directory as `dj_local_conf.json` or at your root directory as a hidden file. This notebook walks you through the setup process.

In [ ]:
import datajoint as dj

## Setup - Credentials

Now let's set up the host, user and password in the `dj.config` global variable

In [ ]:
dj.conn()

## Setup - `dj.config['custom']`

The major component of the current workflow is the [DataJoint Calcium Imaging Element](https://github.com/datajoint/element-array-ephys). Calcium Imaging Element requires configurations in the field `custom` in `dj.config`:

### Database prefix

Giving a prefix to schema could help on the configuration of privilege settings. For example, if we set prefix `neuro_`, every schema created with the current workflow will start with `neuro_`, e.g. `neuro_lab`, `neuro_subject`, `neuro_scan` etc.

The prefix could be configured in `dj.config` as follows. CodeBook users should keep their username as the prefix for schema declaration permissions.

In [ ]:
username_as_prefix = dj.config["database.user"] + "_"
dj.config["custom"] = {"database.prefix": username_as_prefix}

### Root directories for raw/processed data

`imaging_root_data_dir` field indicates the root directory for
+ The **raw data** from ScanImage or Scanbox (e.g. `*.tif`)
+ The processed results from Suite2p or CaImAn (e.g. `F.npy`). 

This can be specific to each machine. The root path typically **does not** contain information of subjects or sessions, all data from subjects/sessions should be subdirectories in the root path.

+ In the example dataset downloaded with [these instructions](00-data-download-optional.ipynb), `/tmp/test_data` will be the root. 

The example dataset is attached to this GitHub Codespace environment, so the root directory will be:

In [ ]:
dj.config["custom"]["imaging_root_data_dir"] = "/tmp/example_data"  # local download

## Save configuration

We could save this as a file, either as a local json file, or a global file. Local configuration file is saved as `dj_local_conf.json` in current directory, which is great for project-specific settings.

For first-time and CodeBook users, we recommend saving globally. This will create a hidden configuration file saved in your root directory, loaded whenever there is no local version to override it.

In [ ]:
dj.config.save_local()

## Activate DataJoint Elements

+ The current workflow is composed of multiple database schemas, each of them corresponds to a module within `workflow_calcium_imaging.pipeline`

In [ ]:
from workflow_calcium_imaging.pipeline import lab, subject, session, scan, imaging, Equipment

## Workflow diagram

This workflow is assembled from 4 DataJoint elements:
+ [element-lab](https://github.com/datajoint/element-lab)
+ [element-animal](https://github.com/datajoint/element-animal)
+ [element-session](https://github.com/datajoint/element-session)
+ [element-calcium-imaging](https://github.com/datajoint/element-calcium-imaging)


In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(scan)
    + dj.Diagram(imaging)
)

## Insert entries

In [ ]:
subject.Subject.insert1(
    dict(
        subject="subject1",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="ScanImage acquisition. Suite2p processing.",
    )
)

In [ ]:
Equipment.insert1(dict(scanner="ScanImage"))

In [ ]:
session_key = dict(subject="subject1", session_datetime="2021-04-30 12:22:15.032")

session.Session.insert1(session_key)

In [ ]:
session.SessionDirectory.insert1(
    dict(**session_key,
        session_dir="subject1/session1")
)

In [ ]:
scan.Scan.insert1(
    dict(
        **session_key,
        scan_id=0,
        scanner="ScanImage",
        acq_software="ScanImage",
        scan_notes="",
    )
)

## Populate tables

In [ ]:
populate_settings = {
                    "display_progress": True,
                    "reserve_jobs": False,
                    "suppress_errors": False,
                    }

In [ ]:
scan.ScanInfo.populate(**populate_settings)

### Define Suite2p parameters

In [ ]:
params_suite2p = {
    "look_one_level_down": 0.0,
    "fast_disk": [],
    "delete_bin": False,
    "mesoscan": False,
    "h5py": [],
    "h5py_key": "data",
    "save_path0": [],
    "subfolders": [],
    "nplanes": 1,
    "nchannels": 1,
    "functional_chan": 1,
    "tau": 1.0,
    "fs": 10.0,
    "force_sktiff": False,
    "preclassify": 0.0,
    "save_mat": False,
    "combined": True,
    "aspect": 1.0,
    "do_bidiphase": False,
    "bidiphase": 0.0,
    "do_registration": True,
    "keep_movie_raw": False,
    "nimg_init": 300,
    "batch_size": 500,
    "maxregshift": 0.1,
    "align_by_chan": 1,
    "reg_tif": False,
    "reg_tif_chan2": False,
    "subpixel": 10,
    "smooth_sigma": 1.15,
    "th_badframes": 1.0,
    "pad_fft": False,
    "nonrigid": True,
    "block_size": [128, 128],
    "snr_thresh": 1.2,
    "maxregshiftNR": 5.0,
    "1Preg": False,
    "spatial_hp": 50.0,
    "pre_smooth": 2.0,
    "spatial_taper": 50.0,
    "roidetect": True,
    "sparse_mode": False,
    "diameter": 12,
    "spatial_scale": 0,
    "connected": True,
    "nbinned": 5000,
    "max_iterations": 20,
    "threshold_scaling": 1.0,
    "max_overlap": 0.75,
    "high_pass": 100.0,
    "inner_neuropil_radius": 2,
    "min_neuropil_pixels": 350,
    "allow_overlap": False,
    "chan2_thres": 0.65,
    "baseline": "maximin",
    "win_baseline": 60.0,
    "sig_baseline": 10.0,
    "prctile_baseline": 8.0,
    "neucoeff": 0.7,
    "xrange": np.array([0, 0]),
    "yrange": np.array([0, 0]),
}

In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=params_suite2p,
    paramset_desc="Calcium imaging analysis with Suite2p using default Suite2p parameters",
)

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        processing_output_dir="subject1/session1/suite2p",
    )
)

In [ ]:
imaging.Processing.populate(**populate_settings)

In [ ]:
imaging.Curation.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time="2021-04-30 12:22:15.032",
        curation_output_dir="subject1/session1/suite2p",
        manual_curation=False,
        curation_note="",
    )
)

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

In [ ]:
imaging.Segmentation.populate(**populate_settings)

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

In [ ]:
imaging.Activity.populate(**populate_settings)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Query and fetch data

+ DataJoint provides functions to query data and fetch.  For a detailed tutorials, visit our [general tutorial site](https://playground.datajoint.io/).

+ Running through the pipeline, we have ingested data of subject3 into the database.

In [ ]:
session_key = (session.Session & 'subject = "subject1"').fetch("KEY")[0]

### `imaging.ProcessingParamSet`, `imaging.ProcessingTask`, `imaging.Processing`, and `imaging.Curation` tables

+ The parameters used for Suite2p or CaImAn are stored in `imaging.ProcessingParamSet` under a `paramset_idx`.

+ The processing details for Suite2p and CaImAn are stored in `imaging.ProcessingTask` and `imaging.Processing` for the utilized `paramset_idx`.

+ After the motion correction and segmentation, the results may go through a curation process. 
    
    + If it did not go through curation, a copy of the `imaging.ProcessingTask` entry is inserted into `imaging.Curation` with the `curation_output_dir` identical to the `processing_output_dir`.

    + If it did go through a curation, a new entry will be inserted into `imaging.Curation`, with a `curation_output_dir` specified.

    + `imaging.Curation` supports multiple curations of an entry in `imaging.ProcessingTask`.

In [ ]:
imaging.ProcessingParamSet()

In [ ]:
imaging.ProcessingTask * imaging.Processing & session_key

In this example workflow, `curation_output_dir` is the same as the `processing_output_dir`, as these results were not manually curated.

In [ ]:
imaging.Curation & session_key

### `imaging.MotionCorrection` table

+ After processing and curation, results are passed to the `imaging.MotionCorrection` and `imaging.Segmentation` tables.

+ For the example data, the raw data is corrected with rigid and non-rigid motion correction which is stored in `imaging.MotionCorrection.RigidMotionCorrection` and `imaging.MotionCorrection.NonRigidMotionCorrection`, respectively. 

+ Lets first query the information for one curation.

In [ ]:
curation_key = (imaging.Curation & session_key & "curation_id=0").fetch1("KEY")

In [ ]:
curation_key

In [ ]:
imaging.MotionCorrection.RigidMotionCorrection & curation_key

In [ ]:
imaging.MotionCorrection.NonRigidMotionCorrection & curation_key

+ For non-rigid motion correction, the details for the individual blocks are stored in `imaging.MotionCorrection.Block`.

In [ ]:
imaging.MotionCorrection.Block & curation_key & "block_id=0"

+ Summary images are stored in `imaging.MotionCorrection.Summary`

    + Reference image - image used as an alignment template

    + Average image - mean of registered frames

    + Correlation image - correlation map (computed during region of interest \[ROI\] detection)

    + Maximum projection image - max of registered frames

In [ ]:
imaging.MotionCorrection.Summary & curation_key & "field_idx=0"

+ Lets fetch the `average_image` and plot it.

In [ ]:
average_image = (
    imaging.MotionCorrection.Summary & curation_key & "field_idx=0"
).fetch1("average_image")

In [ ]:
plt.imshow(average_image);

### `imaging.Segmentation` table

+ Lets fetch and plot a mask stored in the `imaging.Segmentation.Mask` table for one `curation_id`.

+ Each mask can be associated with a field by the attribute `mask_center_z`.  For example, masks with `mask_center_z=0` are in the field identified with `field_idx=0` in `scan.ScanInfo.Field`.

In [ ]:
mask_xpix, mask_ypix = (
    imaging.Segmentation.Mask * imaging.MaskClassification.MaskType
    & curation_key
    & "mask_center_z=0"
    & "mask_npix > 130"
).fetch("mask_xpix", "mask_ypix")

In [ ]:
mask_image = np.zeros(np.shape(average_image), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    mask_image[ypix, xpix] = True

In [ ]:
plt.imshow(average_image)
plt.contour(mask_image, colors="white", linewidths=0.5);

### `imaging.MaskClassification` table

+ This table provides the `mask_type` and `confidence` for the mask classification.

In [ ]:
imaging.MaskClassification.MaskType & curation_key & "mask=0"

### `imaging.Fluorescence` and `imaging.Activity` tables

+ Lets fetch and plot the flourescence and activity traces for one mask.

In [ ]:
query_cells = (
    imaging.Segmentation.Mask * imaging.MaskClassification.MaskType
    & curation_key
    & "mask_center_z=0"
    & "mask_npix > 130"
).proj()

In [ ]:
fluorescence_traces = (imaging.Fluorescence.Trace & query_cells).fetch(
    "fluorescence", order_by="mask"
)

activity_traces = (imaging.Activity.Trace & query_cells).fetch(
    "activity_trace", order_by="mask"
)

sampling_rate = (scan.ScanInfo & curation_key).fetch1("fps")  # [Hz]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax2 = ax.twinx()

for f, a in zip(fluorescence_traces, activity_traces):
    ax.plot(np.r_[: f.size] * 1 / sampling_rate, f, "k", label="fluorescence trace")
    ax2.plot(
        np.r_[: a.size] * 1 / sampling_rate,
        a,
        "r",
        alpha=0.5,
        label="deconvolved trace",
    )

    break

ax.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)

ax.legend(loc="upper left", prop={"size": 14})
ax2.legend(loc="upper right", prop={"size": 14})

ax.set_xlabel("Time (s)")
ax.set_ylabel("Activity (a.u.)")
ax2.set_ylabel("Activity (a.u.)");

# Drop schemas

+ Schemas are not typically dropped in a production workflow with real data in it. 
+ At the developmental phase, it might be required for the table redesign.
+ When dropping all schemas is needed, the following is the dependency order.

In [ ]:
from workflow_calcium_imaging.pipeline import *

In [ ]:
# imaging.schema.drop()
# scan.schema.drop()
# session.schema.drop()
# subject.schema.drop()
# lab.schema.drop()